# **Template OP on salish**

In [23]:
%matplotlib inline

import sys
import xarray as xr
import numpy as np
import os
import math
from datetime import datetime, timedelta
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ErrorCode, ParcelsRandom

sys.path.append('/home/jvalenti/MOAD/analysis-jose/notebooks/parcels')
from Kernels_beaching import DeleteParticle, Buoyancy, AdvectionRK4_3D, Stokes_drift, Beaching
from OP_functions_beaching import *

In [24]:
# Define paths
local = 0 #Set to 0 when working on server
paths = path(local)

In [25]:
Dat=xr.open_dataset(get_WW3_path(datetime(2018, 12, 23)))
path_NEMO = make_prefix(datetime(2018, 12, 23), paths['NEMO'])
Dat0=xr.open_dataset(path_NEMO + '_grid_W.nc')
Dat=xr.open_dataset(path_NEMO + '_grid_T.nc')
coord=xr.open_dataset(paths['coords'],decode_times=False)
WW3 = xr.open_dataset(get_WW3_path(datetime(2018, 12, 23)))
batt=xr.open_dataset(paths['mask'],decode_times=False)

## Define and save mask for distance to coast

In [26]:
# def maskcoast(ilat,jlon):
#     baty = batt.mbathy[0,:,:]
#     maski = 0
#     if baty[ilat,jlon]>1e-1:
#         if baty[ilat+1,jlon] == 0 or baty[ilat+1,jlon+1] == 0 or baty[ilat+1,jlon-1] == 0:
#             maski = 1
#         elif baty[ilat,jlon] == 0 or baty[ilat,jlon+1] == 0 or baty[ilat,jlon-1] == 0:
#             maski = 1
#         elif baty[ilat-1,jlon] == 0 or baty[ilat-1,jlon+1] == 0 or baty[ilat-1,jlon-1] == 0:
#             maski = 1
#     return maski
# maskd = np.zeros(batt.mbathy.shape)
# baty = batt.mbathy[0,:,:]
# for i in range(baty.shape[0]):
#     for j in range(baty.shape[1]):
#         maskd[0,i,j]=maskcoast(i,j)
# batt=xr.open_dataset(paths['mask'],decode_times=False)
# dist=xr.DataArray(attrs={'d':maskd})
# batt['d']=(batt.mbathy.dims,maskd)
# batt.to_netcdf(path='/ocean/jvalenti/MOAD/grid/mesh_maskd201702.nc')

In [27]:
#clat,clon = p_unidist(coord.gphif[0,:,:],coord.glamf[0,:,:],batt.mbathy[0,:,:],10,10)
with open('clat.txt') as f:
    clat = f.read()
    clat= clat[1:-1]
    clat0 = clat.split(",")
    f.close()
with open('clon.txt') as f:
    clon = f.read()
    clon=clon[1:-1]
    clon0 = clon.split(",")
    f.close()
    
clat,clon=[],[]
for i in range(len(clat0)):
    clat.append(float(clat0[i]))
    clon.append(float(clon0[i]))

## Definitions

In [28]:
start = datetime(2018, 10, 1) #Start date
# Set Time length [days] and timestep [seconds]
length = 10 
duration = timedelta(days=length)
dt = 90 #toggle between - or + to pick backwards or forwards
N = len(clat) # number of deploying locations
n = 1 # 1000   # number of particles per location
dmin = list(np.zeros(len(clat))) #minimum depth
dd = 5 #max depth difference from dmin
x_offset, y_offset, zvals = p_deploy(N,n,dmin,dd)

In [29]:
from parcels import Variable

class MPParticle(JITParticle):        
    ro = Variable('ro', initial = 1025)           
    diameter = Variable('diameter', initial = 1.6e-5)
    length = Variable('length', initial = 61e-5)
    Lb = Variable('Lb', initial = 0.0014)  #times needed in days for particle to have 67% probability of beaching if in beaching zone (500m)
    sediment = Variable('sediment', initial = 0)
    beached = Variable('beached', initial = 0)


In [30]:
lon = np.zeros([N,n])
lat = np.zeros([N,n])
for i in range(N):
    lon[i,:]=(clon[i] + x_offset[i,:])
    lat[i,:]=(clat[i] + y_offset[i,:])
z = zvals

In [31]:
#Set start date time and the name of the output file
name = 'beachingL2' #name output file
daterange = [start+timedelta(days=i) for i in range(length)]
fn =  name + '_'.join(d.strftime('%Y%m%d')+'_1n' for d in [start, start+duration]) + '.nc'
outfile = os.path.join(paths['out'], fn)
print(outfile)

/home/jvalenti/MOAD/results/beachingL220181001_1n_20181011_1n.nc


## Simulation

In [32]:
#Fill in the list of variables that you want to use as fields
varlist=['U','V','W','R']
filenames,variables,dimensions=filename_set(start,length,varlist,local)
field_set=FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)

varlist=['US','VS','WL']
filenames,variables,dimensions=filename_set(start,length,varlist,local)

us = Field.from_netcdf(filenames['US'], variables['US'], dimensions,allow_time_extrapolation=True)
vs = Field.from_netcdf(filenames['VS'], variables['VS'], dimensions,allow_time_extrapolation=True)
wl = Field.from_netcdf(filenames['WL'], variables['WL'], dimensions,allow_time_extrapolation=True)
field_set.add_field(us)
field_set.add_field(vs)
field_set.add_field(wl)
field_set.add_vector_field(VectorField("stokes", us, vs, wl))

filenames,variables,dimensions=filename_set(start,length,['Bathy'],local)
Bth = Field.from_netcdf(filenames['Bathy'], variables['Bathy'], dimensions,allow_time_extrapolation=True)
field_set.add_field(Bth)

filenames,variables,dimensions=filename_set(start,length,['D'],local)
Distc = Field.from_netcdf(filenames['D'], variables['D'], dimensions,allow_time_extrapolation=True)
field_set.add_field(Distc)

         It will be opened with no decoding. Filling values might be wrongly parsed.


In [ ]:
# #Load Salish output as fields
#field_set = FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)
pset = ParticleSet.from_list(field_set, MPParticle, lon=lon, lat=lat, depth=z, time=start+timedelta(hours=2))

k_sink = pset.Kernel(Buoyancy)
k_waves = pset.Kernel(Stokes_drift)
k_beach = pset.Kernel(Beaching)

pset.execute(AdvectionRK4_3D + k_sink + k_waves + k_beach,
             runtime=duration, 
             dt=dt,
             output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=1)),
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

INFO: Compiled ArrayMPParticleAdvectionRK4_3DBuoyancyStokes_driftBeaching ==> /tmp/parcels-2894/libba252d6ae0aac11548534427ddcba988_0.so


Particle 1670 lost !! [5400.0, 2.249155692755852, 50.448851484720734, -126.01542965405797]
Particle 1678 lost !! [5400.0, 1.1750005756469706, 50.460732275479245, -126.10717198850483]


INFO: Temporary output files are stored in /home/jvalenti/MOAD/results/out-JETBFWBP.
INFO: You can use "parcels_convert_npydir_to_netcdf /home/jvalenti/MOAD/results/out-JETBFWBP" to convert these to a NetCDF file during the run.
  1% (12600.0 of 864000.0) |             | Elapsed Time: 0:00:09 ETA:   0:16:26

Particle 1671 lost !! [19170.0, 0.14614771491047565, 50.45885432454082, -125.99977427862729]


  1% (16200.0 of 864000.0) |             | Elapsed Time: 0:00:11 ETA:   0:09:23

Particle 1668 lost !! [23310.0, 33.75511754937194, 50.43822200217546, -125.99962234459527]


  8% (72000.0 of 864000.0) |#            | Elapsed Time: 0:00:44 ETA:   0:08:15

Particle 1672 lost !! [77760.0, 4.511737773854587, 50.45425997254297, -125.99984182487364]


100% (864000.0 of 864000.0) |############| Elapsed Time: 0:09:02 Time:  0:09:02
/home/jvalenti/conda_envs/parcels/lib/python3.9/site-packages/numpy/lib/arraysetops.py:270: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)
